In [28]:
using SparseArrays, LightGraphs
include("src/DC.jl")
using .DC

In [29]:
?DC.density_consistency

Density consistency algorithm

Compute approximate marginals of $p( \bf{x} )=\frac1Z \prod_a \psi_{a}(x_{\partial a})$ Input:

  * `Ψ::Vector{<:Factor}`
  * `N::Int`
  * `closure::Symbol = :DC,`									# closure protocol (default = :DC)
  * `maxiter::Int64 = 2000,`
  * `η::Float64 = 1.0,`									# interpolation parameter (DC closure) η=0 gives BP fixed points
  * `γ0::Float64 = 0.0,`									# reinforcement
  * `epsconv::Float64 = 1e-6,`								# precision convergence
  * `Λ::Float64 = 1e-15,`								# add a diagonal matrix Λ*I in the inversion of the full correlation matrix
  * `λ::Float64 = 1e-15,`								# add a diagonal matrix λ*I in the inversion of the cavity and tilted correlation matrices
  * `update::Symbol = :par,`								# type of update (parallel or sequential)
  * `rndamp::Bool = false,`								# eventually, apply a random damping (might be useful in parallel update)
  * `epsclamp::Float64 = 1e-15,`								# clamp
  * `ρ::Float64 = 0.0,`								# damping for parameters' update (ρ = 0 means no damping)
  * `verbose::Bool = true,`								# print at (un)convergence
  * `Hg::Vector{Float64} = zeros(N),`
  * `Ag::Matrix{Float64} = zeros(N,N),`
  * `convtype::Symbol = :params,`						# convergence criterion : tilted moments or gaussian factor parameters
  * `seed :: Int64 = -1,`
  * `callback::Function  = (x...)->nothing,`
  * `h0scale::Float64 = 1e-3,`
  * `state::DCState = DCState( N, [length(Ψ[a].idx) for a=1:length(Ψ)]; h0 = h0scale),`


In [ ]:
# define graph
L = 4;
G = grid([L,L], periodic = true)
Adj = adjacency_matrix(G);
N = L^2

#set couplings
J = Adj.* rand([-1,1],N,N);
#set fields
h = fill(0.1, N);

In [ ]:
β = 1.0;
ψ = DC.IsingFG(N,β*h,β*J, :Fun); # computes moments numerically by running the trace over {-1,1}^2
#ψ = DC.IsingFG(N,β*h,β*J, :PairIsing); # computes moments analytically


#BP -> set η = 0.0 to compute BP fixed points (valid on any graph topology)
#ϕBP , res, it, epsv = density_consistency(ψ,N, epsconv = 1e-10, ρ = 0.9, closure = :DC, verbose = true, η = 0.);

In [ ]:
ϕ , res, it, epsv = density_consistency(ψ,N, epsconv = 1e-5, ρ = 0.95, closure = :DC, verbose = true, Λ = 10., λ = 1e-8, η = 1 - 1e-4, convtype = :params);

In [ ]:
ϕs = DCState(N,[length(ψ[a].idx) for a=1:length(ψ)])


In [ ]:
density_consistency(ψ,N; state  = ϕs,epsconv = 1e-5, ρ = 0.95, closure = :DC, verbose = true, Λ = 10., λ = 1e-8, η = 1.1, convtype = :moments);

In [ ]:
ϕs.Σt

In [ ]:
Float64(19)

In [ ]:
?DC.EnergyFun